In [7]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [8]:
import numpy as np

In [9]:
def build_AlexNET():
    def init_weights(m):
        if type(m) == torch.nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        if type(m) == torch.nn.Conv2d:
            torch.nn.init.xavier_uniform_(m.weight)
        
    net = torch.nn.Sequential(
        # Input: 48x48
    
        torch.nn.Conv2d(in_channels=3, out_channels=96, kernel_size=(11,11),stride=(4,4)), 
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=(3,3), stride=(2,2)), 
        torch.nn.Conv2d(in_channels=96, out_channels=256, kernel_size=(5,5),padding=(2,2)), 
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=(3,3), stride=(2,2)), 
        torch.nn.Conv2d(in_channels=256, out_channels=384, kernel_size=(3,3),padding=(1,1)), 
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels=384, out_channels=384, kernel_size=(3,3),padding=(1,1)), 
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels=384, out_channels=256, kernel_size=(3,3),padding=(1,1)), 
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=(3,3),stride=(2,2)),
        torch.nn.Flatten(),
        
        torch.nn.Linear(5*5*256, 4096),
        torch.nn.ReLU(),
        torch.nn.Dropout(p=0.5),
        torch.nn.Linear(4096, 4096),
        torch.nn.ReLU(),
        torch.nn.Dropout(p=0.5),
        torch.nn.Linear(4096, 1000)
    )
  
    net.apply(init_weights)
    return net


In [10]:
alexnet = build_AlexNET()
print(alexnet)

Sequential(
  (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU()
  (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU()
  (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU()
  (12): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (13): Flatten(start_dim=1, end_dim=-1)
  (14): Linear(in_features=6400, out_features=4096, bias=True)
  (15): ReLU()
  (16): Dropout(p=0.5, inplace=False)
  (17): Linear(in_features=4096, out_features=4096, bias=True)
  (18): ReLU()
  (19): Dropout(p=0.5, inplace=False)
  (20): L

In [ ]:
training_data = datasets.FashionMNIST(
    root="documents/Ne",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

AttributeError: module 'torchvision.datasets' has no attribute 'FashionMNIST'